In [152]:
!pip install lazypredict


In [153]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [154]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [155]:
!wget $fname_url


--2021-12-06 22:18:41--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-12-06 22:18:41--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  --.-KB/s    in 0.1s    

2021-12-06 22:18:42 (155 MB/s) - ‘train.zi

In [156]:
!mkdir "train"
!rm ./train/Train.csv

mkdir: cannot create directory ‘train’: File exists


In [157]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [158]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)

In [ ]:
df.head(5)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.55

In [159]:
def one_hot_encode_df(df,str_index):
    from sklearn.preprocessing import OneHotEncoder
    indf=df.copy(True)
    one_hot=OneHotEncoder()
    col=indf[str_index].values
    col=col.reshape(-1,1)
    one_hot.fit(col)
    out=one_hot.transform(col).toarray()
    codes=one_hot.get_feature_names()
    print(codes)
    for ind,code in enumerate(codes):
        print("Itt:",ind,codes[ind])
        indf[str_index+"_"+code]=out[:,ind]
    return(indf,one_hot)

In [160]:
df,hdl_onehot=one_hot_encode_df(df,"hdl_cholesterol_human")
df,hemo_onehot=one_hot_encode_df(df,"hemoglobin(hgb)_human")
df,col_onehot=one_hot_encode_df(df,"cholesterol_ldl_human")

['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok


In [161]:
df.tail()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
13135,ID_NGPC0DA3,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,...,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93,high,ok,ok,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
13136,ID_XRBUD5U8,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.60,...,1.48,1.46,1.44,1.45,1.43,1.42,1.42,1.41,1.42,1.41,1.39,1.38,1.38,1.38,1.38,1.38,1.38,1.36,1.38,1.38,1.37,1.39,1.38,1.37,1.40,1.39,43.12,19.14,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13137,ID_2M9L5NV2,0.51,0.52,0.52,0.53,0.54,0.54,0.55,0.56,0.57,0.58,0.60,0.61,0.62,0.62,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,...,1.45,1.43,1.43,1.42,1.41,1.42,1.40,1.40,1.39,1.38,1.38,1.36,1.36,1.37,1.37,1.37,1.36,1.37,1.36,1.39,1.39,1.40,1.38,1.43,1.45,1.38,42.48,43.41,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13138,ID_C5V5SD2D,0.46,0.47,0.47,0.47,0.48,0.49,0.49,0.50,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,...,1.29,1.29,1.28,1.27,1.26,1.26,1.25,1.25,1.24,1.23,1.22,1.22,1.22,1.23,1.22,1.21,1.21,1.21,1.22,1.21,1.19,1.18,1.17,1.17,1.15,1.21,41.86,35.10,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13139,ID_9R5ZR9NX,0.50,0.50,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,0.58,...,1.47,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.39,1.38,1.37,1.36,1.37,1.37,1.37,1.37,1.38,1.39,1.38,1.40,1.44,1.35,1.42,42.91,26.30,high,ok,high,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00


In [162]:

del df["hdl_cholesterol_human"]
del df["hemoglobin(hgb)_human"]
del df["cholesterol_ldl_human"]




In [163]:
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np


In [164]:
col_names=df.columns[-9:]

In [165]:
col_names[0]

'hdl_cholesterol_human_x0_high'

In [166]:
X=df.iloc[:,1:-9]
y=df.loc[:,col_names[0]]

In [167]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity
0,0.48,0.48,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.54,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.52,1.51,1.49,1.47,1.46,1.45,1.44,1.42,1.40,1.39,1.38,1.36,1.36,1.34,1.34,1.32,1.32,1.31,1.29,1.29,1.29,1.28,1.27,1.26,1.25,1.25,1.26,1.25,1.24,1.24,1.24,1.23,1.23,1.22,1.21,1.20,1.20,1.18,42.51,34.01
1,0.47,0.47,0.48,0.49,0.49,0.50,0.50,0.51,0.52,0.53,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,0.55,0.55,0.55,...,1.62,1.59,1.58,1.55,1.54,1.53,1.52,1.50,1.49,1.47,1.47,1.45,1.44,1.42,1.42,1.40,1.40,1.38,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.34,1.34,1.33,1.33,1.34,1.33,1.31,1.31,1.30,1.32,1.31,1.26,1.26,44.52,32.09
2,0.44,0.46,0.45,0.46,0.46,0.48,0.48,0.48,0.49,0.51,0.52,0.53,0.54,0.54,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,0.53,0.54,...,1.54,1.54,1.53,1.52,1.50,1.49,1.48,1.46,1.45,1.45,1.44,1.42,1.42,1.40,1.39,1.40,1.39,1.38,1.36,1.35,1.35,1.33,1.33,1.34,1.33,1.34,1.32,1.33,1.33,1.32,1.31,1.30,1.29,1.33,1.27,1.34,1.29,1.35,45.77,24.80
3,0.51,0.51,0.52,0.52,0.52,0.53,0.54,0.55,0.55,0.57,0.58,0.59,0.60,0.60,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,...,1.48,1.47,1.46,1.44,1.42,1.41,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.33,1.32,1.31,1.29,1.29,1.28,1.28,1.27,1.27,1.26,1.25,1.25,1.25,1.24,1.25,1.25,1.25,1.27,1.26,1.29,1.30,1.30,1.31,1.30,1.35,45.84,36.93
4,0.51,0.52,0.52,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,...,1.46,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.35,1.33,1.33,1.32,1.32,1.32,1.30,1.30,1.29,1.30,1.29,1.29,1.29,1.29,1.29,1.30,1.27,1.27,1.27,1.29,1.34,1.32,1.35,38.92,23.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13135,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,0.56,...,1.44,1.43,1.41,1.41,1.39,1.38,1.37,1.36,1.34,1.33,1.33,1.31,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93
13136,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0

In [168]:
offset=int(0.9*len(X))
print(offset)

11826


In [169]:
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,predictions=True)
modells,predictions = clf.fit(X_train, X_test, y_train, y_test)


  3%|▎         | 1/29 [00:14<06:46, 14.52s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.8789954337899544, 'Balanced Accuracy': 0.7313587871982032, 'ROC AUC': 0.7313587871982032, 'F1 Score': 0.8642257053632928, 'Time taken': 14.519981384277344}


 10%|█         | 3/29 [00:33<04:03,  9.38s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9885844748858448, 'Balanced Accuracy': 0.9766598820887142, 'ROC AUC': 0.9766598820887142, 'F1 Score': 0.9885137596652297, 'Time taken': 18.91508674621582}
{'Model': 'BernoulliNB', 'Accuracy': 0.5593607305936074, 'Balanced Accuracy': 0.5630369174620999, 'ROC AUC': 0.5630369174620999, 'F1 Score': 0.6006693008215078, 'Time taken': 0.18052339553833008}


 17%|█▋        | 5/29 [01:12<05:25, 13.58s/it]

...

{'Model': 'SVC', 'Accuracy': 0.8729071537290716, 'Balanced Accuracy': 0.6965995227400337, 'ROC AUC': 0.6965995227400337, 'F1 Score': 0.8502012071464031, 'Time taken': 18.532159090042114}


 97%|█████████▋| 28/29 [02:41<00:07,  7.24s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9931506849315068, 'Balanced Accuracy': 0.9835766423357664, 'ROC AUC': 0.9835766423357664, 'F1 Score': 0.9931082557991379, 'Time taken': 14.866694927215576}


100%|██████████| 29/29 [02:44<00:00,  5.68s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9961948249619482, 'Balanced Accuracy': 0.9908759124087592, 'ROC AUC': 0.9908759124087592, 'F1 Score': 0.9961818682998554, 'Time taken': 3.524498224258423}


In [170]:
modells

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelSpreading,1.00,1.00,1.00,1.00,10.79
KNeighborsClassifier,1.00,1.00,1.00,1.00,0.85
LabelPropagation,1.00,1.00,1.00,1.00,7.92
ExtraTreesClassifier,1.00,1.00,1.00,1.00,2.22
LGBMClassifier,1.00,0.99,0.99,1.00,3.52
XGBClassifier,0.99,0.98,0.98,0.99,14.87
RandomForestClassifier,0.99,0.98,0.98,0.99,13.97
BaggingClassifier,0.99,0.98,0.98,0.99,18.92
ExtraTreeClassifier,0.98,0.97,0.97,0.98,0.18


In [171]:
t1=KNeighborsClassifier()
o=t1.fit(X_train,y_train)

In [172]:
y_train

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
        ... 
11821   0.00
11822   0.00
11823   0.00
11824   0.00
11825   0.00
Name: hdl_cholesterol_human_x0_high, Length: 11826, dtype: float64

In [ ]:
o.predict

array([0., 1., 0., ..., 1., 1., 1.])

In [173]:
from sklearn.neighbors import  KNeighborsClassifier
my_cl=[]

for i in range(9):

    my_cl.append(  KNeighborsClassifier())
    X=df.iloc[:,1:-9]
    y=df.loc[:,col_names[i]]
    my_cl[i].fit(X,y)



In [191]:
X[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [174]:
my_cl

[KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier()]

In [175]:
my_cl[0].predict(X)

array([0., 0., 0., ..., 0., 0., 1.])

In [176]:
my_cl[1].predict(X)

array([0., 0., 0., ..., 0., 0., 0.])

In [177]:
my_cl[2].predict(X)

array([1., 1., 1., ..., 1., 1., 0.])

In [223]:
# submission generation
!rm "./Test.csv"

In [224]:
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv

--2021-12-06 22:59:39--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv [following]
--2021-12-06 22:59:39--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10299206 (9.8M) [text/plain]
Saving to: ‘Test.csv’

Test.csv            100%[===================>]   9.82M  --.-KB/s    in 0.07s   

2021-12-06 22:59:40 (134 MB/s) - ‘Test.csv’ saved 

In [225]:
test_df=pd.read_csv("./Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [226]:
X_test=test_df.iloc[:,1:-3]

In [227]:
X_test[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [228]:
from sklearn.neighbors import  KNeighborsClassifier


for i in range(9):

    y_pred=my_cl[i].predict(X_test)
    test_df[col_names[i]]=y_pred
    



In [229]:
test_df

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
0,ID_37BEI22R,0.45,0.45,0.45,0.46,0.47,0.49,0.49,0.50,0.50,0.51,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.32,1.32,1.31,1.31,1.31,1.30,1.29,1.29,1.29,1.29,1.29,1.27,1.27,1.27,1.26,1.25,1.25,1.25,1.22,1.20,1.21,1.16,1.15,1.17,1.16,1.20,40.88,25.70,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
1,ID_4W85V5DV,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.59,0.60,0.60,...,1.43,1.41,1.40,1.40,1.39,1.38,1.37,1.37,1.36,1.36,1.35,1.35,1.33,1.33,1.34,1.32,1.32,1.32,1.31,1.29,1.27,1.24,1.24,1.22,1.25,1.26,42.35,30.16,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
2,ID_L4YR3NDY,0.44,0.44,0.44,0.45,0.46,0.47,0.47,0.48,0.48,0.50,0.51,0.52,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.51,0.51,0.51,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.51,...,1.25,1.24,1.23,1.22,1.21,1.21,1.20,1.20,1.20,1.19,1.18,1.17,1.17,1.17,1.17,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.18,1.17,42.83,25.51,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
3,ID_U88E3SQ6,0.50,0.51,0.51,0.52,0.53,0.54,0.54,0.54,0.55,0.56,0.57,0.58,0.59,0.59,0.60,0.60,0.60,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.56,0.56,0.56,0.56,0.57,0.57,...,1.29,1.28,1.27,1.26,1.26,1.25,1.24,1.23,1.23,1.21,1.22,1.21,1.21,1.20,1.20,1.20,1.20,1.18,1.17,1.14,1.11,1.05,1.05,1.02,1.03,1.02,40.95,41.32,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00
4,ID_NW7Z3XU7,0.53,0.53,0.54,0.54,0.55,0.56,0.56,0.57,0.58,0.59,0.60,0.62,0.62,0.63,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,0.60,...,1.40,1.39,1.39,1.38,1.36,1.35,1.35,1.34,1.33,1.33,1.33,1.31,1.32,1.30,1.30,1.31,1.31,1.31,1.29,1.29,1.30,1.29,1.27,1.28,1.25,1.29,46.99,24.21,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,ID_ADCBL266,0.51,0.51,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.58,0.58,0.58,0.59,0.59,...,1.47,1.45,1.45,1.43,1.41,1.42,1.40,1.39,1.39,1.38,1.37,1.36,1.36,1.36,1.34,1.35,1.37,1.35,1.37,1.35,1.37,1.36,1.38,1.37,1.35,1.31,35.54,37.52,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
3656,ID_

In [230]:
hdl=test_df.loc[:,"hdl_cholesterol_human_x0_high":	"hdl_cholesterol_human_x0_ok"]
hemo=test_df.loc[:,"hemoglobin(hgb)_human_x0_high":	"hemoglobin(hgb)_human_x0_ok"]
col=test_df.loc[:,"cholesterol_ldl_human_x0_high":"cholesterol_ldl_human_x0_ok"]


In [231]:
hdl_deconv=hdl_onehot.inverse_transform(hdl)
hemo_deconv=hemo_onehot.inverse_transform(hemo)
col_deconv=col_onehot.inverse_transform(col)


In [232]:
test_df["hdl_cholesterol_human"]=hdl_deconv
test_df["hemoglobin(hgb)_human"]=hemo_deconv
test_df["cholesterol_ldl_human"]=col_deconv


In [233]:
!rm SampleSubmission.csv
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv

--2021-12-06 23:00:29--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv [following]
--2021-12-06 23:00:30--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 384318 (375K) [text/plain]
Saving to: ‘SampleSubmission.csv’

SampleSubmission.cs 100%[===================>] 375.31K  --.-KB/s    in 0.01s   

2021

In [234]:
submission_df=pd.read_csv("./SampleSubmission.csv")

In [237]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [243]:
outdf=test_df
outdf.set_index("Reading_ID",inplace=True)

In [244]:
outdf.loc["ID_UPO13B7Z"]

absorbance0                     0.49
absorbance1                     0.50
absorbance2                     0.50
absorbance3                     0.51
absorbance4                     0.52
                                ... 
hemoglobin(hgb)_human_x0_low    0.00
hemoglobin(hgb)_human_x0_ok     1.00
cholesterol_ldl_human_x0_high   0.00
cholesterol_ldl_human_x0_low    0.00
cholesterol_ldl_human_x0_ok     1.00
Name: ID_UPO13B7Z, Length: 184, dtype: object

In [245]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [246]:
for i in submission_df.index:
    #print(i)
    indexer=submission_df.loc[i,"Reading_ID"]

    splitter=indexer.split("_",2)
    out_index=splitter[0]+"_"+splitter[1]
    col_index=splitter[2]
    print(out_index,col_index)
    result=outdf.loc[out_index][col_index]
    submission_df.iloc[i,1]=result
submission_df.set_index("Reading_ID", inplace=True)
def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)
submission_fname=create_submission_name("BloodAI")
submission_fname
'BloodAI_submission_20211205_171730.csv'
submission_df.to_csv(submission_fname,index=True)

A streamkimeneten csak az utolsó 5000 sor látható.
ID_JVY48D8U cholesterol_ldl_human
ID_JVY48D8U hemoglobin(hgb)_human
ID_JW4F66XM cholesterol_ldl_human
ID_JW4F66XM hdl_cholesterol_human
ID_JW4F66XM hemoglobin(hgb)_human
ID_JWXML3KU cholesterol_ldl_human
ID_JWXML3KU hemoglobin(hgb)_human
ID_JWXML3KU hdl_cholesterol_human
ID_JXCWA9OK cholesterol_ldl_human
ID_JXCWA9OK hemoglobin(hgb)_human
ID_JXCWA9OK hdl_cholesterol_human
ID_JY2WRCO4 hemoglobin(hgb)_human
ID_JY2WRCO4 hdl_cholesterol_human
ID_JY2WRCO4 cholesterol_ldl_human
ID_JYFHLCK2 cholesterol_ldl_human
ID_JYFHLCK2 hemoglobin(hgb)_human
ID_JYFHLCK2 hdl_cholesterol_human
ID_JYUWNU78 cholesterol_ldl_human
ID_JYUWNU78 hemoglobin(hgb)_human
ID_JYUWNU78 hdl_cholesterol_human
ID_JYV38755 hemoglobin(hgb)_human
ID_JYV38755 cholesterol_ldl_human
ID_JYV38755 hdl_cholesterol_human
ID_JYYFUL4N cholesterol_ldl_human
ID_JYYFUL4N hdl_cholesterol_human
ID_JYYFUL4N hemoglobin(hgb)_human
ID_JZ2BEIAY cholesterol_ldl_human
ID_JZ2BEIAY hdl_cholesterol_hum